#QAGNN

although some codes are run here to see the data. we cannot run with google colab env because the ram size is too small

this colab exist just as a reference

In [1]:
!git clone https://github.com/michiyasunaga/qagnn.git

Cloning into 'qagnn'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 85 (delta 29), reused 15 (delta 15), pack-reused 40
Unpacking objects: 100% (85/85), done.


In [4]:
%cd qagnn

/content/qagnn


## Downloading dependency and QAdataset


In [5]:
!pip install torch==1.8.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers==3.4.0
!pip install nltk spacy==2.1.6
!python -m spacy download en

# for torch-geometric
!pip install torch-scatter==2.0.7 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-sparse==0.6.9 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install torch-geometric==1.7.0 -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5 MB 14 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0+cu101 which is incompatible.
     |████████████████████████████████| 1.3 MB 3.7 MB/s 
     |████████████████████████████████| 2.9 MB 44.8 MB/s 
     |████████████████████████████████| 880 kB 38.9 MB/s 
     |████████████████████████████████| 1.2 MB 40.8 MB/s

In [9]:
!chmod +x download_preprocessed_data.sh
!./download_preprocessed_data.sh

--2022-05-20 06:58:23--  https://nlp.stanford.edu/projects/myasu/QAGNN/data_preprocessed_release.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4226838278 (3.9G) [application/zip]
Saving to: ‘data_preprocessed_release.zip’

data_preprocessed_r 100%[===================>]   3.94G  21.1MB/s    in 3m 19s  

2022-05-20 07:01:43 (20.2 MB/s) - ‘data_preprocessed_release.zip’ saved [4226838278/4226838278]

Archive:  data_preprocessed_release.zip
   creating: data_preprocessed_release/
   creating: data_preprocessed_release/cpnet/
  inflating: data_preprocessed_release/cpnet/conceptnet.en.unpruned.graph  
  inflating: data_preprocessed_release/cpnet/conceptnet.en.csv  
  inflating: data_preprocessed_release/cpnet/matcher_patterns.json  
  inflating: data_preprocessed_release/cpnet/conceptnet-assertions-5.6.0.csv  
  inflating: data_preproc

## statement jsonl example:

answerkey, question:{choices, stem}

statement and question pair:


```
{"answerKey": "A", "id": "075e483d21c29a511267ef62bedc0461", "question": {"question_concept": "punishing", "choices": [{"label": "A", "text": "ignore"}, {"label": "B", "text": "enforce"}, {"label": "C", "text": "authoritarian"}, {"label": "D", "text": "yell at"}, {"label": "E", "text": "avoid"}], "stem": "The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?"}, "statements": [{"label": true, "statement": "The sanctions against the school were a punishing blow, and they seemed to ignore the efforts the school had made to change."}, {"label": false, "statement": "The sanctions against the school were a punishing blow, and they seemed to enforce the efforts the school had made to change."}, {"label": false, "statement": "The sanctions against the school were a punishing blow, and they seemed to authoritarian the efforts the school had made to change."}, {"label": false, "statement": "The sanctions against the school were a punishing blow, and they seemed to yell at the efforts the school had made to change."}, {"label": false, "statement": "The sanctions against the school were a punishing blow, and they seemed to avoid the efforts the school had made to change."}]}
{"answerKey": "B", "id": "61fe6e879ff18686d7552425a36344c8", "question": {"question_concept": "people", "choices": [{"label": "A", "text": "race track"}, {"label": "B", "text": "populated areas"}, {"label": "C", "text": "the desert"}, {"label": "D", "text": "apartment"}, {"label": "E", "text": "roadblock"}], "stem": "Sammy wanted to go to where the people were.  Where might he go?"}, "statements": [{"label": false, "statement": "Sammy wanted to go to where the people were.  race track might he go."}, {"label": true, "statement": "Sammy wanted to go to where the people were.  populated areas might he go."}, {"label": false, "statement": "Sammy wanted to go to where the people were.  the desert might he go."}, {"label": false, "statement": "Sammy wanted to go to where the people were.  apartment might he go."}, {"label": false, "statement": "Sammy wanted to go to where the people were.  roadblock might he go."}]}
{"answerKey": "A", "id": "4c1cb0e95b99f72d55c068ba0255c54d", "question": {"question_concept": "choker", "choices": [{"label": "A", "text": "jewelry store"}, {"label": "B", "text": "neck"}, {"label": "C", "text": "jewlery box"}, {"label": "D", "text": "jewelry box"}, {"label": "E", "text": "boutique"}], "stem": "To locate a choker not located in a jewelry box or boutique where would you go?"}, "statements": [{"label": true, "statement": "To locate a choker not located in a jewelry box or boutique jewelry store would you go."}, {"label": false, "statement": "To locate a choker not located in a jewelry box or boutique neck would you go."}, {"label": false, "statement": "To locate a choker not located in a jewelry box or boutique jewlery box would you go."}, {"label": false, "statement": "To locate a choker not located in a jewelry box or boutique jewelry box would you go."}, {"label": false, "statement": "To locate a choker not located in a jewelry box or boutique boutique would you go."}]}

```


where stem is question
```
{"answerKey": "D", "id": "e6766a66f8d326bf00fbf628a0e4ef24", "question": {"question_concept": "marsh", "choices": [{"label": "A", "text": "low lands"}, {"label": "B", "text": "new york"}, {"label": "C", "text": "forest"}, {"label": "D", "text": "louisiana"}, {"label": "E", "text": "everglades"}], "stem": "Bill is stuck in marsh when a man comes up to him peaking Cajun, where is he?"}}
{"answerKey": "E", "id": "a76403b4921a9281b6ee2a7241a5ec9f", "question": {"question_concept": "grill", "choices": [{"label": "A", "text": "backyard"}, {"label": "B", "text": "restaurant"}, {"label": "C", "text": "crockpot"}, {"label": "D", "text": "neighbor's house"}, {"label": "E", "text": "barbeque"}], "stem": "What is it called when you slowly cook using a grill?"}}

```



## run training

open run_qagnn__csqa.sh and remove line about conda

In [15]:
!chmod +x run_qagnn__obqa.sh
!./run_qagnn__obqa.sh

***** hyperparameters *****
dataset: obqa
enc_name: roberta-large
batch_size: 128
learning_rate: elr 1e-5 dlr 1e-3
gnn: dim 200 layer 5
******************************
tcmalloc: large alloc 3273826304 bytes == 0xaa70000 @  0x7f9bdb07e1e7 0x7f9b866da0ce 0x7f9b86731e57 0x7f9b86732a6f 0x7f9b867d8c5d 0x593835 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x5118f8 0x549e0e 0x593fce 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e
tcmalloc: large alloc 3273826304 bytes == 0xcdc9a000 @  0x7f9bdb07e1e7 0x7f9b866da0ce 0x7f9b86730cf5 0x7f9b867d986d 0x7f9b867da17f 0x7f9b867da2d0 0x4bc4ab 0x7f9b8671b944 0x59371f 0x515244 0x549576 0x593fce 0x548ae9 0x51566f 0x549e0e 0x593fce 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7f9bdac7bc87 0x5b621a
tcmalloc: large alloc 3273826304 bytes == 0x190ec4000 @  0x7f9bdb060b6b 0x7f9bdb080379 0x7f9b8725225e 0x7f9b87253

In [16]:
#!/bin/bash

# export CUDA_VISIBLE_DEVICES=0,1
# dt=`date '+%Y%m%d_%H%M%S'`


# dataset="csqa"
# model='roberta-large'
# shift
# shift
# args=$@


# elr=
# dlr="1e-3"
# bs=64
# mbs=2
# n_epochs=15
# num_relation=38 #(17 +2) * 2: originally 17, add 2 relation types (QA context -> Q node; QA context -> A node), and double because we add reverse edges


# k=5 #num of gnn layers
# gnndim=200

# echo "***** hyperparameters *****"
# echo "dataset: $dataset"
# echo "enc_name: $model"
# echo "batch_size: $bs"
# echo "learning_rate: elr $elr dlr $dlr"
# echo "gnn: dim $gnndim layer $k"
# echo "******************************"

# save_dir_pref='saved_models'
# mkdir -p $save_dir_pref
# mkdir -p logs

! python3 -u qagnn.py --dataset csqa \
      --encoder roberta-large -k 5 --gnn_dim 200 -elr "1e-5" -dlr "1e-3" -bs 64 -mbs 2 --fp16 true --seed 0 \
      --num_relation 38 \
      --n_epochs 15 --max_epochs_before_stop 10  \
      --train_adj data/csqa/graph/train.graph.adj.pk \
      --dev_adj   data/csqa/graph/dev.graph.adj.pk \
      --test_adj  data/csqa/graph/test.graph.adj.pk \
      --train_statements  data/csqa/statement/train.statement.jsonl \
      --dev_statements  data/csqa/statement/dev.statement.jsonl \
      --test_statements  data/csqa/statement/test.statement.jsonl \
      --save_model \
      --save_dir saved_models/csqa/enc-roberta-large__k5__gnndim200__bs64__seed0__34 \
  


90b0b2573887
pid: 835
screen: 

gpu: 

Namespace(att_head_num=2, batch_size=64, cuda=True, dataset='csqa', debug=False, decoder_lr=0.001, dev_adj='data/csqa/graph/dev.graph.adj.pk', dev_statements='data/csqa/statement/dev.statement.jsonl', drop_partial_batch=False, dropoutf=0.2, dropoutg=0.2, dropouti=0.2, encoder='roberta-large', encoder_layer=-1, encoder_lr=1e-05, ent_emb=['tzw'], ent_emb_paths=['data/cpnet/tzw.ent.npy'], eval_batch_size=2, fc_dim=200, fc_layer_num=0, fill_partial_batch=False, fp16=True, freeze_ent_emb=True, gnn_dim=200, inhouse=True, inhouse_train_qids='data/csqa/inhouse_split_qids.txt', init_range=0.02, k=5, load_model_path=None, log_interval=10, loss='cross_entropy', lr_schedule='fixed', max_epochs_before_stop=10, max_grad_norm=1.0, max_node_num=200, max_seq_len=100, mini_batch_size=2, mode='train', n_epochs=15, num_relation=38, optim='radam', refreeze_epoch=10000, save_dir='saved_models/csqa/enc-roberta-large__k5__gnndim200__bs64__seed0__34', save_model=True, see

In [13]:
!chmod +x eval_qagnn__csqa.sh
!./eval_qagnn__csqa.sh

******************************
dataset: csqa
******************************
90b0b2573887
pid: 471
screen: 

gpu: 0

tcmalloc: large alloc 3273826304 bytes == 0xc23a000 @  0x7fb30b0181e7 0x7fb2b66740ce 0x7fb2b66cbe57 0x7fb2b66cca6f 0x7fb2b6772c5d 0x593835 0x548c51 0x5127f1 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x5118f8 0x593dd7 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fb30ac15c87
tcmalloc: large alloc 3273826304 bytes == 0xcf464000 @  0x7fb30b0181e7 0x7fb2b66740ce 0x7fb2b66cacf5 0x7fb2b677386d 0x7fb2b677417f 0x7fb2b67742d0 0x4bc4ab 0x7fb2b66b5944 0x59371f 0x515244 0x549576 0x593fce 0x548ae9 0x51566f 0x593dd7 0x5118f8 0x593dd7 0x5118f8 0x549576 0x604173 0x5f5506 0x5f8c6c 0x5f9206 0x64faf2 0x64fc4e 0x7fb30ac15c87 0x5b621a
tcmalloc: large alloc 3273826304 bytes == 0x19268e000 @  0x7fb30affab6b 0x7fb30b01a379 0x7fb2b71ec25e 0x7fb2b71ed9d2 0x7fb2f4a90443 0x7fb2f4318263 0x7fb2f48193f7 0x7fb